<a href="https://colab.research.google.com/github/pawanraje16/CVDLlab/blob/main/mango_alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [ ]:
# Directories and hyperparameters
train_dir = "/content/dataset/Mango_leaf_disease1/train"
valid_dir = "/content/dataset/Mango_leaf_disease1/val"
test_dir = "/content/dataset/Mango_leaf_disease1/test"
batch_size = 32
img_size = (224, 224)
num_epochs = 8

In [ ]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations and loaders
transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
num_classes = len(train_dataset.classes)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Load AlexNet with no default weights but allow loading pretrained weights
model = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)  # Load pretrained weights

# Modify the classifier for the correct number of classes
num_classes = 10  # Ensure this matches your dataset
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

print("Model is ready!")


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:02<00:00, 115MB/s]


Model is ready!


In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.2f}%")

Epoch 1/8 - Loss: 0.2846 - Accuracy: 90.25%
Epoch 2/8 - Loss: 0.0171 - Accuracy: 99.43%
Epoch 3/8 - Loss: 0.0115 - Accuracy: 99.64%
Epoch 4/8 - Loss: 0.0784 - Accuracy: 98.18%
Epoch 5/8 - Loss: 0.0207 - Accuracy: 99.36%
Epoch 6/8 - Loss: 0.0064 - Accuracy: 99.71%
Epoch 7/8 - Loss: 0.0028 - Accuracy: 99.89%
Epoch 8/8 - Loss: 0.0069 - Accuracy: 99.82%


In [ ]:
# Validation loop
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in valid_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

val_acc = 100.0 * correct / total
print(f"Validation Accuracy: {val_acc:.2f}%")

# Save the fine-tuned model
torch.save(model.state_dict(), "AlexNet_finetuned.pth")
print("Model saved as AlexNet_finetuned.pth")

Validation Accuracy: 99.00%
Model saved as AlexNet_finetuned.pth


In [ ]:
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

test_acc = 100.0 * correct / total
print(f"Test Accuracy: {test_acc:.2f}%")

Test Accuracy: 99.75%
